In [1]:
import os
import boto3
import pickle
import numpy
import music_tag
import random
import json
from pathlib import Path

from music_utils import *

import numpy as np
from pymilvus import connections, utility
from pymilvus import Collection, DataType, FieldSchema, CollectionSchema

In [2]:
from dotenv import load_dotenv
load_dotenv()

URI = os.getenv("MILVUS_URI")
TOKEN = os.getenv("MILVUS_TOKEN")
BUCKET = os.getenv("BUCKET")

DATASET = "Music_Dataset"

In [3]:
def get_list_of_all_filepaths_in_S3(bucket):
    s3_client = boto3.client('s3')
    try:
        contents = s3_client.list_objects(Bucket=bucket)
        return [content['Key'] for content in contents['Contents']]

    except Exception as e:
        pass
    return None

In [4]:
s3_client = boto3.client('s3') 

list_of_files_in_S3 = get_list_of_all_filepaths_in_S3(BUCKET)
print(len(list_of_files_in_S3))

174


In [5]:
import random
rdm_from_s3 = random.choice(list_of_files_in_S3)
print(rdm_from_s3)

uploads/Music_Dataset/Funkadelic/[02]_Can_you_get_to_that.mp3


In [ ]:
connections.connect("default",
                    uri=URI,
                    token=TOKEN)
print(f"Connecting to DB: {URI}")
print(utility.list_collections())

collection_512 = Collection("embeddings_512")

In [7]:
query_path = rdm_from_s3.replace("uploads/", "")

In [8]:
entities = collection_512.query(
    expr=f'path == "{query_path}"',
    output_fields=["*"]
)
embedding_512 = entities[0]['embedding']

In [9]:
query_artist = entities[0]['artist']
query_album = entities[0]['album']
query_title = entities[0]['title']
query_top3_genres = entities[0]['top_5_genres'][:3]

print(query_artist)
print(query_album)
print(query_title)
print(query_top3_genres)

Funkadelic
Maggot brain
Can you get to that
['pop', 'alternative', 'rock']


In [10]:
bigresult = collection_512.search(
    data=[embedding_512],
    anns_field="embedding",
    param={"nprobe": 16},
    limit=200,
    offset=1,
    output_fields=["*"],
)

print(f'{"Title":<30} | {"Artist":<25} | {"Top 5 Genres":<25} | {"Path":<50}')
print('-' * 145)

already_proposed_artits = []
count = 0
for result in bigresult[0]:
    if count >= 20:
        break
    already_proposed_artits.append(result.artist)
    count += 1
    print(f"{result.title[:29]:<30} | {result.artist[:39]:<25} | {', '.join(result.top_5_genres[:2]):<25} | {result.path[:49]:<50} | ")

Title                          | Artist                    | Top 5 Genres              | Path                                              
-------------------------------------------------------------------------------------------------------------------------------------------------
Rehab                          | Amy Winehouse             | pop, rock                 | Music_Dataset/Amy_Whinehouse/01_Rehab.mp3          | 
Hit it and quit it             | Funkadelic                | rock, alternative         | Music_Dataset/Funkadelic/[03]_Hit_it_and_quit_it.  | 
Come to Me                     | The Black Seeds           | reggae, dub               | Music_Dataset/The_Black_Seeds/01-the_black_seeds-  | 
this land is your land         | Sharon Jones and the Dap-Kings | pop, rock                 | Music_Dataset/Sharon_Jones/Sharon_Jones_and_the_D  | 
Take Your Changes              | The Black Seeds           | reggae, rock              | Music_Dataset/The_Black_Seeds/03-the_black_seeds